In [ ]:
import os
import shutil
import subprocess
import time

In [ ]:
def copy(folder, dst):
    shutil.copytree(folder, f"{dst}/{folder}")

In [ ]:
os.chdir("../..")

In [ ]:
base_dir = "notebooks/Microservices/images"
procs = []

In [ ]:
def build(app):
    name = app.split("/")[-1]
    subprocess.run(
        ["docker", "build", "-q", "-t", f"rsys/{name}", f"{base_dir}/{name}"]
    )

In [ ]:
def run(app, port):
    name = app.split("/")[-1]
    return subprocess.Popen(
        [
            "docker",
            "run",
            "--cpus",
            "1",
            "-m",
            "1g",
            "-p",
            f"{port}:8080",
            f"rsys/{name}",
        ]
    )

In [ ]:
# fetch
app = f"{base_dir}/fetch_media_lists"
if os.path.exists(app):
    shutil.rmtree(app)
shutil.copytree("notebooks/Microservices/fetch_media_lists", app)
copy("notebooks/API/API", app)
copy("environment/animeplanet", app)
copy("environment/kitsu", app)
copy("environment/mal", app)
copy("environment/proxies", app)
build(app)
procs.append(run(app, 3000))

In [ ]:
# compress
app = f"{base_dir}/compress_media_lists"
if os.path.exists(app):
    shutil.rmtree(app)
shutil.copytree("notebooks/Microservices/compress_media_lists", app)
copy("notebooks/ImportDatasets", app)
copy("notebooks/ProcessData", app)
copy("data/processed_data", app)
build(app)
procs.append(run(app, 3001))

In [ ]:
while True:
    time.sleep(3600)

In [ ]:
for p in procs:
    p.terminate()
for p in procs:
    p.wait()